In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bureau-v2/bureau_nouveau.csv
/kaggle/input/previous-applications-merged/previous_merged.csv
/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/bureau-and-balance/bureau (1).csv
/kaggle/input/bureau-handcrafted/bureau_handcrafted.csv


In [2]:
df_application = pd.read_csv("/kaggle/input/home-credit-default-risk/application_train.csv")
df_bureau_balance = pd.read_csv("/kaggle/input/bureau-handcrafted/bureau_handcrafted.csv")
df_previous = pd.read_csv("/kaggle/input/previous-applications-merged/previous_merged.csv")

y = df_application[['TARGET', 'SK_ID_CURR']]
y=y.astype(int)
df_application=df_application.drop('TARGET',axis=1)


y.shape
df_application.shape



(307511, 121)

In [3]:
# Now encoding the categorical values using labelencoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import OrdinalEncoder

encoder = OrdinalEncoder(handle_unknown="value", handle_missing="value") 

categorical_columns = df_application.select_dtypes(exclude = 'number').columns
df_categorical = df_application[categorical_columns].copy()
df_categorical = pd.DataFrame(encoder.fit_transform(df_categorical),columns = df_categorical.columns)

numerical_columns = df_application.select_dtypes(include='number').columns
df_numerical = df_application[numerical_columns].copy()

df = pd.concat([df_numerical,df_categorical],axis=1)
df

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,1,1,1,1,1,1,1,1,1,1
1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,2,2,1,2,2,2,1,1,2,1
2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,1,1,1,1,2,3,5,4,8,3
3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,1,3,1,1,1,1,5,4,8,3
4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,1,1,1,2,3,4,5,4,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327,-236,-8456.0,...,1,5,3,6,3,18,1,1,1,1
307507,456252,0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775,365243,-4388.0,...,1,4,1,19,2,6,1,1,1,1
307508,456253,0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966,-7921,-6737.0,...,2,5,1,4,3,2,1,1,3,1
307509,456254,0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961,-4786,-2562.0,...,1,2,1,1,1,28,5,1,1,1


In [4]:
# impute numerical missing values by first finding out the columns where more than 40% of rows are empty

# Step 1 : Drop useless features

empty_columns=[]

for column in df.columns:
    if (df[column].isnull().sum()/307511)>0.4 or (df[column].dropna().var() / (df[column].abs().mean() + 1e-9))<0.01:
        empty_columns.append(column)

df= df.drop(columns=empty_columns,axis=1)

print(empty_columns)
print(len(empty_columns))

['REGION_POPULATION_RELATIVE', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'TOTALAREA_MODE']
48


In [5]:
df.isnull().sum()

SK_ID_CURR              0
CNT_CHILDREN            0
AMT_INCOME_TOTAL        0
AMT_CREDIT              0
AMT_ANNUITY            12
                       ..
ORGANIZATION_TYPE       0
FONDKAPREMONT_MODE      0
HOUSETYPE_MODE          0
WALLSMATERIAL_MODE      0
EMERGENCYSTATE_MODE     0
Length: 73, dtype: int64

In [6]:
# Step 2 : Impute remaining features
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df = pd.DataFrame(imp_mean.fit_transform(df),columns = df.columns)
df.shape

(307511, 73)

In [7]:
aggregated_bureau = df_bureau_balance.groupby('SK_ID_CURR').agg(
    max_days_credit = ('DAYS_CREDIT', 'max'),
    max_overdue = ('CREDIT_DAY_OVERDUE', 'max'),
    max_prolong = ('CNT_CREDIT_PROLONG', 'max'),
    max_credit_update = ('DAYS_CREDIT_UPDATE', 'max'),
    active_credit = ('CREDIT_ACTIVE', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    type_credit = ('CREDIT_TYPE', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    max_credit_end = ('CREDIT_END_COUNT', 'min'),
    late_credit = ('pct_late', 'max'),
    very_late_credit = ('pct_very_late', 'max'),
    on_time_credit = ('pct_on_time', 'min')
)
print(aggregated_bureau.shape)

aggregated_bureau['SK_ID_CURR'] = aggregated_bureau.index
aggregated_bureau = aggregated_bureau.reset_index(drop=True)
aggregated_bureau.head()

(134542, 10)


,max_days_credit,max_overdue,max_prolong,max_credit_update,active_credit,type_credit,max_credit_end,late_credit,very_late_credit,on_time_credit,SK_ID_CURR
0,-49.0,0.0,0.0,-6.0,1.0,1.0,-1.0,0.076923,0.0,0.022222,100001.0
1,-103.0,0.0,0.0,-7.0,1.0,1.0,0.0,0.500000,0.0,0.187500,100002.0
2,-62.0,0.0,0.0,-11.0,2.0,1.0,-5.0,0.000000,0.0,0.583333,100005.0
3,-1138.0,0.0,0.0,-18.0,1.0,1.0,210.0,0.000000,0.0,0.277778,100010.0
4,-1210.0,0.0,0.0,-4.0,1.0,1.0,-489.0,0.045455,0.0,0.000000,100013.0


In [8]:
###########################
# Seeing correlation between features and dropping the similar one 
# Seeing correlation between features and target and keeping the most important ones
import seaborn as sns
import matplotlib.pyplot as plt

features = df.copy()
features = features.drop('SK_ID_CURR',axis=1)


#fig, ax = plt.subplots(figsize=(10, 6))

#sns.heatmap(features.corr(), ax=ax, annot=True)

correlation = features.corr()
correlation

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,...,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
CNT_CHILDREN,1.000000,0.012882,0.002145,0.021374,-0.001826,0.330938,-0.239818,0.183395,-0.028019,0.240714,...,0.021773,-0.016911,0.018452,-0.169901,-0.001399,0.053150,0.025241,0.036776,0.038490,0.038409
AMT_INCOME_TOTAL,0.012882,1.000000,0.156870,0.191657,0.159600,0.027261,-0.064223,0.027805,0.008506,0.063994,...,0.063806,-0.009363,-0.002326,-0.047047,0.000686,0.007545,-0.037663,-0.049713,-0.042993,-0.050129
AMT_CREDIT,0.002145,0.156870,1.000000,0.770127,0.986588,-0.055436,-0.066838,0.009621,-0.006575,0.065519,...,0.065614,-0.030500,-0.028338,-0.042308,-0.005303,0.002513,-0.045555,-0.057572,-0.046802,-0.058121
AMT_ANNUITY,0.021374,0.191657,0.770127,1.000000,0.774661,0.009445,-0.104331,0.038513,0.011268,0.103533,...,0.085417,-0.052223,-0.017100,-0.070703,-0.005935,0.001170,-0.052918,-0.066997,-0.052787,-0.068084
AMT_GOODS_PRICE,-0.001826,0.159600,0.986588,0.774661,1.000000,-0.053416,-0.064826,0.011561,-0.009262,0.063471,...,0.074475,-0.033297,-0.031582,-0.040528,-0.005422,0.003960,-0.044572,-0.061954,-0.051685,-0.062686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ORGANIZATION_TYPE,0.053150,0.007545,0.002513,0.001170,0.003960,0.149042,-0.196692,0.051900,0.061696,0.196409,...,0.019393,-0.029900,0.013681,-0.183086,-0.001571,1.000000,0.000395,-0.000084,0.000788,0.000187
FONDKAPREMONT_MODE,0.025241,-0.037663,-0.045555,-0.052918,-0.044572,0.016419,0.005318,0.051107,0.009394,-0.004982,...,-0.050316,-0.014374,-0.007651,-0.002732,0.000914,0.000395,1.000000,0.649039,0.625084,0.618987
HOUSETYPE_MODE,0.036776,-0.049713,-0.057572,-0.066997,-0.061954,0.019396,0.010295,0.068681,0.014891,-0.009908,...,-0.073151,-0.024057,-0.018218,-0.003309,-0.000724,-0.000084,0.649039,1.000000,0.908404,0.937047
WALLSMATERIAL_MODE,0.038490,-0.042993,-0.046802,-0.052787,-0.051685,0.020755,0.008710,0.068355,0.015400,-0.008297,...,-0.064843,-0.024709,-0.009247,-0.002289,-0.001535,0.000788,0.625084,0.908404,1.000000,0.892869


In [9]:
features = []
for col in correlation.columns:
    index_label = correlation[(correlation[col] > 0.7) & (correlation[col] != 1)].index.tolist()
    if index_label :
        index_label.append(col)
        features.append((index_label,col))

features

for group in features:
    for feature in group[0]:
        print("| Feature 1: ",group[1], "  ",feature," | Feature 2")


| Feature 1:  CNT_CHILDREN    CNT_FAM_MEMBERS  | Feature 2
| Feature 1:  CNT_CHILDREN    CNT_CHILDREN  | Feature 2
| Feature 1:  AMT_CREDIT    AMT_ANNUITY  | Feature 2
| Feature 1:  AMT_CREDIT    AMT_GOODS_PRICE  | Feature 2
| Feature 1:  AMT_CREDIT    AMT_CREDIT  | Feature 2
| Feature 1:  AMT_ANNUITY    AMT_CREDIT  | Feature 2
| Feature 1:  AMT_ANNUITY    AMT_GOODS_PRICE  | Feature 2
| Feature 1:  AMT_ANNUITY    AMT_ANNUITY  | Feature 2
| Feature 1:  AMT_GOODS_PRICE    AMT_CREDIT  | Feature 2
| Feature 1:  AMT_GOODS_PRICE    AMT_ANNUITY  | Feature 2
| Feature 1:  AMT_GOODS_PRICE    AMT_GOODS_PRICE  | Feature 2
| Feature 1:  DAYS_EMPLOYED    NAME_INCOME_TYPE  | Feature 2
| Feature 1:  DAYS_EMPLOYED    DAYS_EMPLOYED  | Feature 2
| Feature 1:  CNT_FAM_MEMBERS    CNT_CHILDREN  | Feature 2
| Feature 1:  CNT_FAM_MEMBERS    CNT_FAM_MEMBERS  | Feature 2
| Feature 1:  REGION_RATING_CLIENT    REGION_RATING_CLIENT_W_CITY  | Feature 2
| Feature 1:  REGION_RATING_CLIENT    REGION_RATING_CLIENT  | 

In [10]:
# Combiner bureau + application_train

df = pd.merge(df,aggregated_bureau,on='SK_ID_CURR',how='left')
df


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,max_days_credit,max_overdue,max_prolong,max_credit_update,active_credit,type_credit,max_credit_end,late_credit,very_late_credit,on_time_credit
0,100002.0,0.0,202500.0,406597.5,24700.5,351000.0,-9461.0,-637.0,-3648.0,-2120.0,...,-103.0,0.0,0.0,-7.0,1.0,1.0,0.0,0.5,0.0,0.187500
1,100003.0,0.0,270000.0,1293502.5,35698.5,1129500.0,-16765.0,-1188.0,-1186.0,-291.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004.0,0.0,67500.0,135000.0,6750.0,135000.0,-19046.0,-225.0,-4260.0,-2531.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006.0,0.0,135000.0,312682.5,29686.5,297000.0,-19005.0,-3039.0,-9833.0,-2437.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007.0,0.0,121500.0,513000.0,21865.5,513000.0,-19932.0,-3038.0,-4311.0,-3458.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251.0,0.0,157500.0,254700.0,27558.0,225000.0,-9327.0,-236.0,-8456.0,-1982.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252.0,0.0,72000.0,269550.0,12001.5,225000.0,-20775.0,365243.0,-4388.0,-4090.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253.0,0.0,153000.0,677664.0,29979.0,585000.0,-14966.0,-7921.0,-6737.0,-5150.0,...,-713.0,0.0,0.0,-5.0,1.0,1.0,387.0,0.0,0.0,0.387097
307509,456254.0,0.0,171000.0,370107.0,20205.0,319500.0,-11961.0,-4786.0,-2562.0,-931.0,...,-1104.0,0.0,0.0,-401.0,1.0,1.0,0.0,0.0,0.0,0.216216


In [11]:
print(df.shape)
print(y.shape)
print(df.isnull().sum())
#df = df[df['type_credit'].notna()]

df = pd.merge(df,df_previous,on='SK_ID_CURR',how=('left'))
#df = df[df['SK_DPD_DEF_y'].notna()]
df.shape

(307511, 83)
(307511, 2)
SK_ID_CURR               0
CNT_CHILDREN             0
AMT_INCOME_TOTAL         0
AMT_CREDIT               0
AMT_ANNUITY              0
                     ...  
type_credit         215280
max_credit_end      215280
late_credit         215280
very_late_credit    215280
on_time_credit      215280
Length: 83, dtype: int64


(307511, 130)

In [12]:
df.isnull().sum()
#df = df[df['late_credit'].notna()]
df.isnull().sum()

#y = y[y['SK_ID_CURR'].isin(df['SK_ID_CURR'])]
print(y.shape)
print(df.shape)
y=y.drop('SK_ID_CURR',axis=1)

imp_mean3 = SimpleImputer(missing_values=np.nan,strategy='median')
df  = pd.DataFrame(imp_mean3.fit_transform(df),columns=df.columns)
print(df.isnull().sum())
df.head()

(307511, 2)
(307511, 130)
SK_ID_CURR                0
CNT_CHILDREN              0
AMT_INCOME_TOTAL          0
AMT_CREDIT_x              0
AMT_ANNUITY_x             0
                         ..
SK_DPD_DEF_y              0
NUM_INSTALMENT_VERSION    0
NUM_INSTALMENT_NUMBER     0
DAYS_LATE_PAYMENT         0
MISSING_AMT_PAYMENT       0
Length: 130, dtype: int64


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD_y,SK_DPD_DEF_y,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_LATE_PAYMENT,MISSING_AMT_PAYMENT
0,100002.0,0.0,202500.0,406597.5,24700.5,351000.0,-9461.0,-637.0,-3648.0,-2120.0,...,25877.019789,21.0,21.0,1.0,0.0,0.0,0.0,1.0,33.0,3375.0
1,100003.0,0.0,270000.0,1293502.5,35698.5,1129500.0,-16765.0,-1188.0,-1186.0,-291.0,...,25877.019789,21.0,21.0,1.0,0.0,0.0,0.0,1.0,33.0,3375.0
2,100004.0,0.0,67500.0,135000.0,6750.0,135000.0,-19046.0,-225.0,-4260.0,-2531.0,...,25877.019789,21.0,21.0,1.0,0.0,0.0,0.0,1.0,33.0,3375.0
3,100006.0,0.0,135000.0,312682.5,29686.5,297000.0,-19005.0,-3039.0,-9833.0,-2437.0,...,0.000000,6.0,6.0,1.0,0.0,0.0,1.0,1.0,77.0,0.0
4,100007.0,0.0,121500.0,513000.0,21865.5,513000.0,-19932.0,-3038.0,-4311.0,-3458.0,...,25877.019789,21.0,21.0,1.0,0.0,0.0,0.0,1.0,33.0,3375.0


In [13]:
df.shape

(307511, 130)

In [14]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(df,y,test_size=0.2)

x_train = x_train.drop('SK_ID_CURR', axis=1)
x_test = x_test.drop('SK_ID_CURR', axis=1)

In [15]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier


'''import xgboost as xgb

model = xgb.XGBClassifier(
    n_estimators=2000,
    max_depth=40,
    max_leaves=60,
    colsample_bytree=0.2,
    subsample=1.0,
    learning_rate=1.0,
    booster='gbtree',
    num_parallel_tree=2000,
    tree_method='hist',    # Use hist + device='cuda'
    device='cuda',         # Specify GPU device here
    objective='binary:logistic',
    verbosity=1,
    random_state=42,
    scale_pos_weight = y_train['TARGET'].value_counts()[0] / y_train['TARGET'].value_counts()[1]  # balanced class weights

)

'''
feature_names = x_train.columns.tolist()

# Initialize model with banking-optimized parameters
model = RandomForestClassifier(
    n_estimators=2000,
    criterion='gini',
    min_samples_split=50,        # Prevents overfitting to small segments
    min_samples_leaf=20,         # Good for financial data stability
    max_features=0.2,            # Feature sampling for diversity
    max_depth=40,                # Deep trees to capture complex patterns
    max_leaf_nodes=60,           # Controls model complexity
    class_weight='balanced',     # Crucial for imbalanced banking data
    n_jobs=-1,                   # Use all cores for faster training
    random_state=42              # For reproducibility
)

# Train model
model.fit(x_train, y_train.values.ravel())
"""
# Get and display feature importances
importances = model.feature_importances_
sorted_idx = importances.argsort()[::-1]  # Descending order

# Print top 60 features with importance scores
print("Top 60 Predictive Features:")
for idx in sorted_idx[:60]:
    print(f"{feature_names[idx]}: {importances[idx]:.6f}")

# Get top feature names for potential feature selection
#top_features = [feature_names[i] for i in sorted_idx[:60]]
"""
# Predict and evaluate
y_predict = model.predict(x_test)
accuracy = accuracy_score(y_test.values.ravel(), y_predict)
print(f"\nModel Accuracy: {accuracy:.4f}")



Model Accuracy: 0.6882


In [16]:

# Import application_test

df_test = pd.read_csv("/kaggle/input/home-credit-default-risk/application_test.csv")
df_test = df_test.drop(columns=empty_columns,axis=1)
df_test.head()
print(df_test.shape)
df_test

(48744, 73)


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [17]:

test_categorical_columns = df_test.select_dtypes(exclude='number').columns
test_df_categorical = df_test[test_categorical_columns].copy()
test_df_categorical = pd.DataFrame(encoder.transform(test_df_categorical),columns=test_categorical_columns)

test_numerical_columns = df_test.select_dtypes(include = 'number').columns
test_df_numerical = df_test[test_numerical_columns].copy()

In [18]:
# Imputing the numerical values test dataset
df_test = pd.concat([test_df_numerical,test_df_categorical],axis=1)

df_test = pd.DataFrame(imp_mean.transform(df_test),columns = df_test.columns)

In [19]:
df_test

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,100001.0,0.0,135000.0,568800.0,20560.5,450000.0,-19241.0,-2329.0,-5170.0,-812.0,...,2.0,2.0,1.0,19.0,7.0,14.0,5.0,1.0,1.0,1.0
1,100005.0,0.0,99000.0,222768.0,17370.0,180000.0,-18064.0,-4469.0,-9118.0,-1623.0,...,1.0,2.0,1.0,14.0,6.0,10.0,5.0,4.0,8.0,3.0
2,100013.0,0.0,202500.0,663264.0,69777.0,630000.0,-20038.0,-4458.0,-2175.0,-3503.0,...,2.0,2.0,1.0,5.0,2.0,26.0,5.0,4.0,8.0,3.0
3,100028.0,2.0,315000.0,1575000.0,49018.5,1575000.0,-13976.0,-1866.0,-2000.0,-4208.0,...,1.0,2.0,1.0,6.0,1.0,1.0,1.0,1.0,3.0,1.0
4,100038.0,1.0,180000.0,625500.0,32067.0,625500.0,-13040.0,-2191.0,-4000.0,-4262.0,...,1.0,2.0,1.0,19.0,6.0,1.0,5.0,4.0,8.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221.0,0.0,121500.0,412560.0,17473.5,270000.0,-19970.0,-5169.0,-9094.0,-3399.0,...,1.0,4.0,1.0,19.0,1.0,5.0,5.0,4.0,8.0,3.0
48740,456222.0,2.0,157500.0,622413.0,31909.5,495000.0,-11186.0,-1149.0,-3015.0,-3003.0,...,1.0,2.0,1.0,6.0,2.0,15.0,5.0,4.0,8.0,3.0
48741,456223.0,1.0,202500.0,315000.0,33205.5,315000.0,-15922.0,-3037.0,-2681.0,-1504.0,...,1.0,2.0,1.0,19.0,1.0,1.0,5.0,1.0,1.0,1.0
48742,456224.0,0.0,225000.0,450000.0,25128.0,450000.0,-13968.0,-2731.0,-1461.0,-1364.0,...,2.0,2.0,1.0,4.0,2.0,10.0,5.0,1.0,3.0,1.0


In [20]:
df_test = pd.merge(df_test,aggregated_bureau,on='SK_ID_CURR',how='left')
df_test = pd.merge(df_test,df_previous,on='SK_ID_CURR',how='left')

In [21]:
df_test

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD_y,SK_DPD_DEF_y,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_LATE_PAYMENT,MISSING_AMT_PAYMENT
0,100001.0,0.0,135000.0,568800.0,20560.5,450000.0,-19241.0,-2329.0,-5170.0,-812.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100005.0,0.0,99000.0,222768.0,17370.0,180000.0,-18064.0,-4469.0,-9118.0,-1623.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100013.0,0.0,202500.0,663264.0,69777.0,630000.0,-20038.0,-4458.0,-2175.0,-3503.0,...,18101.079844,96.0,96.0,1.0,1.0,1.0,0.0,4.0,38.0,23147.82
3,100028.0,2.0,315000.0,1575000.0,49018.5,1575000.0,-13976.0,-1866.0,-2000.0,-4208.0,...,7968.609184,49.0,49.0,1.0,0.0,0.0,0.0,4.0,19.0,8505.00
4,100038.0,1.0,180000.0,625500.0,32067.0,625500.0,-13040.0,-2191.0,-4000.0,-4262.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221.0,0.0,121500.0,412560.0,17473.5,270000.0,-19970.0,-5169.0,-9094.0,-3399.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48740,456222.0,2.0,157500.0,622413.0,31909.5,495000.0,-11186.0,-1149.0,-3015.0,-3003.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223.0,1.0,202500.0,315000.0,33205.5,315000.0,-15922.0,-3037.0,-2681.0,-1504.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48742,456224.0,0.0,225000.0,450000.0,25128.0,450000.0,-13968.0,-2731.0,-1461.0,-1364.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
imp_mean2 = SimpleImputer(missing_values=np.nan, strategy='mean')
df_test.isnull().sum()
df_test = pd.DataFrame(imp_mean2.fit_transform(df_test),columns=df_test.columns)
df_test.isnull().sum()

df_test.head()
df_test.index=df_test['SK_ID_CURR']
df_test = df_test.drop('SK_ID_CURR', axis=1)
df_test

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,...,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD_y,SK_DPD_DEF_y,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_LATE_PAYMENT,MISSING_AMT_PAYMENT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001.0,0.0,135000.0,568800.0,20560.5,450000.0,-19241.0,-2329.0,-5170.0,-812.0,1.0,...,62481.091755,36.028563,36.028563,1.028318,12.41682,0.989174,0.560367,2.648991,41.432416,9390.988993
100005.0,0.0,99000.0,222768.0,17370.0,180000.0,-18064.0,-4469.0,-9118.0,-1623.0,1.0,...,62481.091755,36.028563,36.028563,1.028318,12.41682,0.989174,0.560367,2.648991,41.432416,9390.988993
100013.0,0.0,202500.0,663264.0,69777.0,630000.0,-20038.0,-4458.0,-2175.0,-3503.0,1.0,...,18101.079844,96.000000,96.000000,1.000000,1.00000,1.000000,0.000000,4.000000,38.000000,23147.820000
100028.0,2.0,315000.0,1575000.0,49018.5,1575000.0,-13976.0,-1866.0,-2000.0,-4208.0,1.0,...,7968.609184,49.000000,49.000000,1.000000,0.00000,0.000000,0.000000,4.000000,19.000000,8505.000000
100038.0,1.0,180000.0,625500.0,32067.0,625500.0,-13040.0,-2191.0,-4000.0,-4262.0,1.0,...,62481.091755,36.028563,36.028563,1.028318,12.41682,0.989174,0.560367,2.648991,41.432416,9390.988993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456221.0,0.0,121500.0,412560.0,17473.5,270000.0,-19970.0,-5169.0,-9094.0,-3399.0,1.0,...,62481.091755,36.028563,36.028563,1.028318,12.41682,0.989174,0.560367,2.648991,41.432416,9390.988993
456222.0,2.0,157500.0,622413.0,31909.5,495000.0,-11186.0,-1149.0,-3015.0,-3003.0,1.0,...,62481.091755,36.028563,36.028563,1.028318,12.41682,0.989174,0.560367,2.648991,41.432416,9390.988993
456223.0,1.0,202500.0,315000.0,33205.5,315000.0,-15922.0,-3037.0,-2681.0,-1504.0,1.0,...,62481.091755,36.028563,36.028563,1.028318,12.41682,0.989174,0.560367,2.648991,41.432416,9390.988993


In [23]:
#temp2 = df_test['SK_ID_CURR']
y_test_predicted = model.predict(df_test)

In [24]:
from pathlib import Path  
# Chemin complet vers le fichier
filepath = Path('/kaggle/working/submission.csv')

columns = df_test.index.astype(str).str.replace('.0', '').astype('Int32')
df_output = pd.DataFrame({
    'SK_ID_CURR': columns,  # REQUIRED column
    'TARGET': y_test_predicted             # Your predictions
})

# Sauvegarde en CSV
df_output.to_csv(filepath, index=False)

import os
print(os.listdir('/kaggle/working/'))  

['__notebook__.ipynb', 'submission.csv']


In [25]:
df_output

,SK_ID_CURR,TARGET
0,100001,1
1,100005,1
2,100013,0
3,100028,0
4,100038,1
...,...,...
48739,456221,0
48740,456222,1
48741,456223,1
48742,456224,0
